In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
df=pd.read_csv('data.csv')

In [3]:
df= df.replace('nan', None, regex=True)

In [9]:
data=df.query("api_spec_id==98849").sort_values(by=['commit_date'])

data['Count2'] = data.apply(lambda x: x['API_changes'] - x['count'] if pd.notnull(x['count']) else None, axis=1)

# add api_changes to Count2 where Count2 is null
data['Count2'] = data.apply(lambda x: x['API_changes'] if pd.isnull(x['Count2']) else x['Count2'], axis=1)

data['commit_date']=pd.to_datetime(data['commit_date'])
data['Year']= data['commit_date'].dt.year
data['Month']= data['commit_date'].dt.month_name()
data['Day']= data['commit_date'].dt.day

first_commit_date = data['commit_date'].min()
data['time_diff_new'] = (data['commit_date'] - first_commit_date).dt.days

data['type1'] = data['count'].apply(lambda x: "Breaking" if pd.notna(x) else None)

main1 = data[['Year','Month','Day','time','info_version','API_changes','type1','count','content']]
main1.fillna('None', inplace=True)
main1

,Year,Month,Day,time,info_version,API_changes,type1,count,content
7257,2016,September,26,07:34,0.1,7.0,None,None,None
7258,2016,October,7,06:54,0.1,15.0,None,None,None
7259,2016,October,13,15:56,0.1,7.0,None,None,None
7260,2016,October,19,07:39,0.1,9.0,None,None,None
7261,2016,October,28,08:11,0.1,9.0,None,None,None
7262,2016,November,15,23:04,0.1,9.0,None,None,None
7263,2016,November,21,13:01,0.1,11.0,Breaking,1.0,Path Removed w/o Deprecation
7264,2016,November,24,09:22,0.1,9.0,Breaking,1.0,Request Parameter Removed
7265,2016,November,28,10:55,0.1,7.0,None,None,None
7266,2017,January,13,09:07,0.1,9.0,Breaking,2.0,Request Parameter Became Required


In [10]:
api1 = main1.values.tolist()

In [11]:
api1 = [[None if val == 'None' else val for val in inner_lst] for inner_lst in api1]

In [12]:
def create_hierarchy(api1):
    root = {"name": "root", "children": []}
    for row in api1:
        year, month, day, time, info_version, api_changes, type1, count, content = row + [None]* (9 - len(row))
        year_node = None
        for node in root["children"]:
            if node["name"] == year:
                year_node = node
                break
        if not year_node:
            year_node = {"name": year, "children": []}
            root["children"].append(year_node)
        month_node = None
        for node in year_node["children"]:
            if node["name"] == month:
                month_node = node
                break
        if not month_node:
            month_node = {"name": month, "children": []}
            year_node["children"].append(month_node)
        day_node = None
        for node in month_node["children"]:
            if node["name"] == str(day):
                day_node = node
                break
        if not day_node:
            day_node = {"name": str(day), "children": []}
            month_node["children"].append(day_node)
        hour_node = {"name": str(time), "children": []}
        day_node["children"].append(hour_node)
        info_node = {"name": info_version, "value": api_changes, "children": []}
        hour_node["children"].append(info_node)
        if type1:
            type_node = None
            for node in info_node["children"]:
                if node["name"] == type1:
                    type_node = node
                    break
            if not type_node:
                type_node = {"name": type1, "value": count, "children": []}
                info_node["children"].append(type_node)
            if content:
                content_node = {"name": content, "value": count, "children": []}
                type_node["children"].append(content_node)
    return root["children"]


hierarchy1 = create_hierarchy(api1)
print   (hierarchy1)

# copy hierarchy to clipboard
#import pyperclip
#pyperclip.copy(str(hierarchy1))

[{'name': 2016, 'children': [{'name': 'September', 'children': [{'name': '26', 'children': [{'name': '07:34', 'children': [{'name': '0.1', 'value': 7.0, 'children': []}]}]}]}, {'name': 'October', 'children': [{'name': '7', 'children': [{'name': '06:54', 'children': [{'name': '0.1', 'value': 15.0, 'children': []}]}]}, {'name': '13', 'children': [{'name': '15:56', 'children': [{'name': '0.1', 'value': 7.0, 'children': []}]}]}, {'name': '19', 'children': [{'name': '07:39', 'children': [{'name': '0.1', 'value': 9.0, 'children': []}]}]}, {'name': '28', 'children': [{'name': '08:11', 'children': [{'name': '0.1', 'value': 9.0, 'children': []}]}]}]}, {'name': 'November', 'children': [{'name': '15', 'children': [{'name': '23:04', 'children': [{'name': '0.1', 'value': 9.0, 'children': []}]}]}, {'name': '21', 'children': [{'name': '13:01', 'children': [{'name': '0.1', 'value': 11.0, 'children': [{'name': 'Breaking', 'value': 1.0, 'children': [{'name': 'Path Removed w/o Deprecation', 'value': 1.0,